In [596]:
from typing import Any, Union
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 500)

# PREPROCESS

In [597]:
train = pd.read_csv('data/original/train.csv')
test = pd.read_csv('data/original/test.csv')

In [598]:
train.head()

,ID,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,Прекращение работы по болезни,Сахарный диабет,Гепатит,Онкология,Хроническое заболевание легких,Бронжиальная астма,Туберкулез легких,ВИЧ/СПИД,Регулярный прим лекарственных средств,Травмы за год,Переломы,Статус Курения,Возраст курения,Сигарет в день,Пассивное курение,Частота пасс кур,Алкоголь,Возраст алког,Время засыпания,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы",ID_y,Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца
0,54-102-358-02,М,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,0,0,0,0,0,0,0,0,0,0,0,Курит,15.0,20.0,0,NaN,употребляю в настоящее время,18.0,22:00:00,06:00:00,0,0,0,54-102-358-02,0,0,0,0,0
1,54-103-101-01,Ж,в разводе,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,0,1,0,0,0,0,0,0,1,0,1,Никогда не курил(а),NaN,NaN,0,NaN,никогда не употреблял,NaN,00:00:00,04:00:00,1,0,0,54-103-101-01,1,1,0,0,0
2,54-501-026-03,Ж,в браке в настоящее время,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,0,0,0,0,0,0,0,0,1,0,0,Никогда не курил(а),NaN,NaN,1,1-2 раза в неделю,употребляю в настоящее время,17.0,23:00:00,07:00:00,0,0,0,54-501-026-03,0,0,0,0,0
3,54-501-094-02,М,в браке в настоящее время,европейская,Русские,Атеист / агностик,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,0,0,0,1,0,0,0,0,1,0,0,Бросил(а),12.0,10.0,1,3-6 раз в неделю,употребляю в настоящее время,13.0,23:00:00,07:00:00,0,0,0,54-501-094-02,1,0,0,0,0
4,54-503-022-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,операторы и монтажники установок и машинного о...,0,0,1,1,1,0,0,0,0,0,1,0,1,Никогда не курил(а),NaN,NaN,1,не менее 1 раза в день,употребляю в настоящее время,16.0,23:00:00,06:00:00,0,0,0,54-503-022-01,1,0,1,1,0


In [599]:
train = train.drop('ID_y', axis=1)

In [600]:
CAT_UNORDERED_COLS = [
    'Пол', 'Семья', 'Этнос', 'Национальность', 'Религия', 'Образование', 'Профессия', 'Статус Курения',
    'Частота пасс кур', 'Алкоголь', 'Время засыпания', 'Время пробуждения'
]
CAT_ORDERED_COLS = ['Образование_поряд', 'Статус Курения_поряд', 'Частота пасс кур_поряд']
BINARY_COLS = [
    'Вы работаете?', 'Выход на пенсию', 'Прекращение работы по болезни', 'Сахарный диабет',
    'Гепатит', 'Онкология', 'Хроническое заболевание легких', 'Бронжиальная астма',
    'Туберкулез легких ', 'ВИЧ/СПИД', 'Регулярный прим лекарственных средств', 'Травмы за год', 'Переломы',
    'Пассивное курение', 'Сон после обеда', 'Спорт, клубы', 'Религия, клубы',  
    ]
REAL_COLS = [
    'Возраст курения', 'Сигарет в день', 'Возраст алког', 'Время засыпания_поряд', 
    'Время пробуждения_поряд']



In [601]:
TARGETS = [
    'Сердечная недостаточность', 'Стенокардия, ИБС, инфаркт миокарда', 
    'Прочие заболевания сердца', 'ОНМК', 'Артериальная гипертензия']

In [602]:
def add_features(data: pd.DataFrame) -> pd.DataFrame:
    # образование как порядковый признак
    data['Образование_поряд'] = data['Образование'].str.slice(0, 1).astype(np.int8)
    data['Статус Курения_поряд'] = (
        data['Статус Курения'].replace({
            'Никогда не курил(а)': 0,
            'Никогда не курил': 0,
            'Бросил(а)': 1,
            'Курит': 2
        }))
    data['Частота пасс кур_поряд'] = (
        data['Частота пасс кур'].replace({
            '1-2 раза в неделю': 0,
            '3-6 раз в неделю': 1,
            'не менее 1 раза в день': 2,
            '2-3 раза в день': 3,
            '4 и более раз в день': 4
        })
    )
    data['Алкоголь_поряд'] = (
        data['Алкоголь'].replace({
            'никогда не употреблял': 0,
            'ранее употреблял': 1,
            'употребляю в настоящее время': 2
        })
    )


    def process_sleep_time(s: pd.Series) -> pd.Series:
        s = pd.to_datetime(s)
        date = pd.Timestamp(s.iloc[0].date())
        mask = s < (date + pd.Timedelta(hours=12))
        s.loc[mask] = s.loc[mask] + pd.Timedelta(days=1)
        s = (s - date) / pd.Timedelta(hours=1)
        return s

    data['Время засыпания_поряд'] = process_sleep_time(data['Время засыпания'])


    def process_wakeup_time(s: pd.Series) -> pd.Series:
        s = pd.to_datetime(s)
        date = pd.Timestamp(s.iloc[0].date())
        return (s - date) / pd.Timedelta(hours=1)

    data['Время пробуждения_поряд'] = process_wakeup_time(data['Время пробуждения'])
    return data

In [603]:
def cast_types(data: pd.DataFrame) -> pd.DataFrame:
    data[REAL_COLS] = data[REAL_COLS].astype(np.float32)
    data[BINARY_COLS] = data[BINARY_COLS].astype(np.int8)
    data[CAT_ORDERED_COLS] = data[CAT_ORDERED_COLS].astype(np.float32)
    data[CAT_UNORDERED_COLS] = data[CAT_UNORDERED_COLS].fillna('NA').astype('category')
    return data

In [604]:
def preprocess(data: pd.DataFrame) -> pd.DataFrame:
    data = add_features(data)
    data = cast_types(data)

    data = data.set_index('ID')
    return data

In [605]:
train = preprocess(train)
test = preprocess(test)

In [606]:
train.to_pickle('data/prepared/train.pkl')
test.to_pickle('data/prepared/test.pkl')

# MODELING

In [607]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import recall_score, f1_score, make_scorer
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.pipeline import *
from sklearn.compose import *
from sklearn.preprocessing import *
from sklearn.svm import *
from sklearn.linear_model import *
from sklearn.neighbors import *
from sklearn.ensemble import *
from sklearn.decomposition import *
from sklearn.impute import *
from sklearn.base import BaseEstimator
from xgboost import XGBClassifier

In [608]:
common_preprocess_pipe = ColumnTransformer(transformers=[
    ('cat_unordered_cols', OneHotEncoder(dtype=np.int8, handle_unknown='ignore', sparse=False), CAT_UNORDERED_COLS),
    ('real_cols', make_pipeline(SimpleImputer(), StandardScaler()), REAL_COLS + CAT_ORDERED_COLS),
    ('binary_cols', 'passthrough', BINARY_COLS)
])

In [609]:
cat_model = CatBoostClassifier(cat_features=CAT_UNORDERED_COLS)

lgb = LGBMClassifier()
lgb_pipe = Pipeline([
    ('common_preprocess_pipe', common_preprocess_pipe),
    ('lgb', lgb)
])

xgb = XGBClassifier()
xgb_pipe = Pipeline([
    ('common_preprocess_pipe', common_preprocess_pipe),
    ('xgb', XGBClassifier())
])

ridge = RidgeClassifierCV()
ridge_pipe = Pipeline([
    ('common_preprocess_pipe', common_preprocess_pipe),
    ('ridge', ridge)
])

log_reg = LogisticRegressionCV(solver='liblinear')
log_reg_pipe = Pipeline([
    ('common_preprocess_pipe', common_preprocess_pipe),
    ('log_reg', log_reg)
])

svm = SVC(probability=True)
svm_pipe = Pipeline([
    ('common_preprocess_pipe', common_preprocess_pipe),
    ('svm', svm)
])

knn = KNeighborsClassifier(n_jobs=-1)
knn_preprocess_pipe = Pipeline([
    ('common_preprocess_pipe', common_preprocess_pipe),
    ('SVD',  TruncatedSVD(n_components=20))
])
knn_pipe = Pipeline([
    ('knn_preprocess_pipe', knn_preprocess_pipe),
    ('knn', knn)
])

rf = RandomForestClassifier(max_depth=20, n_jobs=-1)
rf_pipe = Pipeline([
    ('common_preprocess_pipe', common_preprocess_pipe),
    ('rf', rf)
])

In [610]:
meta_model = MultiOutputClassifier(
    estimator=StackingClassifier(n_jobs=-1,
    # final_estimator=LogisticRegression(),
    passthrough=False,
    estimators=[
        ('cat', cat_model),
        ('log_reg', log_reg_pipe),
        ('svm', svm_pipe),
        ('knn', knn_pipe),
        ('rf', rf_pipe),
        ('lgb', lgb_pipe),
        ('xgb', xgb_pipe),
        ('ridge', ridge_pipe)
    ]
    )
)

## METRICS

In [611]:
def compute_weird_score(y_true: np.ndarray, y_pred: Union[list, np.ndarray]) -> tuple[float, dict]:
    if not isinstance(y_true, np.ndarray):
        y_true = y_true.values

    if isinstance(y_pred, list):
        y_pred = np.hstack([pred[:, 1].reshape(-1, 1) for pred in y_pred])

    avg_metric = 0
    for col in range(y_true.shape[1]):
        max_metric = float('-inf')
        for tresh in np.unique(y_pred[col]):
            curr_metric = recall_score(y_true[col], np.where(y_pred[col] >= tresh, 1, 0), average='macro', zero_division=0)
            if curr_metric > max_metric:
                max_metric = curr_metric
        avg_metric += max_metric
    avg_metric /= y_true.shape[1]
    return avg_metric

In [ ]:
weird_score = make_scorer(score_func=compute_weird_score, greater_is_better=True, needs_proba=True)

In [612]:
def get_tresholds(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> tuple[float, dict]:
    if isinstance(y_pred, list):
        y_pred = np.hstack([pred[:, 1].reshape(-1, 1) for pred in y_pred])
        y_pred = pd.DataFrame(data=y_pred, index=y_true.index, columns=y_true.columns)
    avg_metric = 0
    tresholds = {}
    for col in y_true.columns:
        max_metric = float('-inf')
        for tresh in y_pred[col].unique():
            curr_metric = recall_score(y_true[col].values, np.where(y_pred[col].values >= tresh, 1, 0), average='macro')
            if curr_metric > max_metric:
                max_metric = curr_metric
                tresholds[col] = tresh
        avg_metric += max_metric
    avg_metric /= len(y_true.columns)
    print(avg_metric)
    return tresholds

## CV

In [613]:
cv_results = cross_validate(meta_model, X_train, y_train, scoring=weird_score)

Learning rate set to 0.008348
0:	learn: 0.6855867	total: 1.12ms	remaining: 1.12s
1:	learn: 0.6778329	total: 2.72ms	remaining: 1.36s
2:	learn: 0.6697684	total: 7.58ms	remaining: 2.52s
3:	learn: 0.6622171	total: 9.36ms	remaining: 2.33s
4:	learn: 0.6548967	total: 12.6ms	remaining: 2.51s
5:	learn: 0.6476951	total: 17ms	remaining: 2.82s
6:	learn: 0.6409547	total: 19.1ms	remaining: 2.71s
7:	learn: 0.6337595	total: 21.6ms	remaining: 2.68s
8:	learn: 0.6274281	total: 22.3ms	remaining: 2.45s
9:	learn: 0.6212238	total: 23ms	remaining: 2.28s
10:	learn: 0.6150293	total: 28.7ms	remaining: 2.58s
11:	learn: 0.6090469	total: 31.1ms	remaining: 2.56s
12:	learn: 0.6028704	total: 39ms	remaining: 2.96s
13:	learn: 0.5971547	total: 39.8ms	remaining: 2.8s
14:	learn: 0.5914083	total: 42.3ms	remaining: 2.77s
15:	learn: 0.5859222	total: 42.9ms	remaining: 2.64s
16:	learn: 0.5803674	total: 44.5ms	remaining: 2.57s
17:	learn: 0.5748366	total: 51.3ms	remaining: 2.8s
18:	learn: 0.5695011	total: 56.6ms	remaining: 2.92s


/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:1031: RuntimeWarning: invalid value encountered in true_divide
  * (last_sum / last_over_new_count - new_sum) ** 2


21:	learn: 0.5657890	total: 222ms	remaining: 9.88s
22:	learn: 0.5609963	total: 226ms	remaining: 9.6s
9:	learn: 0.6256132	total: 171ms	remaining: 16.9s
15:	learn: 0.5953764	total: 212ms	remaining: 13s
16:	learn: 0.5903028	total: 212ms	remaining: 12.3s
17:	learn: 0.5853277	total: 225ms	remaining: 12.3s
18:	learn: 0.5804389	total: 225ms	remaining: 11.6s
19:	learn: 0.5756403	total: 226ms	remaining: 11.1s
23:	learn: 0.5565717	total: 255ms	remaining: 10.4s
10:	learn: 0.6197048	total: 199ms	remaining: 17.9s
24:	learn: 0.5522309	total: 259ms	remaining: 10.1s
20:	learn: 0.5704449	total: 244ms	remaining: 11.4s
11:	learn: 0.6137114	total: 214ms	remaining: 17.6s
25:	learn: 0.5477370	total: 274ms	remaining: 10.2s
26:	learn: 0.5435574	total: 276ms	remaining: 9.93s
21:	learn: 0.5630664	total: 295ms	remaining: 13.1s
15:	learn: 0.5939560	total: 281ms	remaining: 17.3s
21:	learn: 0.5651727	total: 268ms	remaining: 11.9s
22:	learn: 0.5605369	total: 273ms	remaining: 11.6s
12:	learn: 0.6083160	total: 236ms	r

/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:1031: RuntimeWarning: invalid value encountered in true_divide
  * (last_sum / last_over_new_count - new_sum) ** 2


16:	learn: 0.6055729	total: 230ms	remaining: 13.3s
17:	learn: 0.6013136	total: 234ms	remaining: 12.8s
16:	learn: 0.6058627	total: 240ms	remaining: 13.9s
20:	learn: 0.5861221	total: 253ms	remaining: 11.8s
17:	learn: 0.6013486	total: 271ms	remaining: 14.8s
18:	learn: 0.5964644	total: 273ms	remaining: 14.1s
21:	learn: 0.5817440	total: 274ms	remaining: 12.2s
22:	learn: 0.5779001	total: 275ms	remaining: 11.7s
23:	learn: 0.5741735	total: 276ms	remaining: 11.2s
19:	learn: 0.5923786	total: 278ms	remaining: 13.6s
20:	learn: 0.5884497	total: 280ms	remaining: 13s
24:	learn: 0.5697270	total: 280ms	remaining: 10.9s
21:	learn: 0.5845183	total: 283ms	remaining: 12.6s
22:	learn: 0.5806331	total: 285ms	remaining: 12.1s
17:	learn: 0.6010276	total: 283ms	remaining: 15.4s
18:	learn: 0.5966264	total: 287ms	remaining: 14.8s
18:	learn: 0.5969689	total: 294ms	remaining: 15.2s
19:	learn: 0.5929451	total: 295ms	remaining: 14.5s
23:	learn: 0.5788140	total: 299ms	remaining: 12.2s
24:	learn: 0.5751243	total: 301ms

In [ ]:
cv_results

{'fit_time': array([6.42726898, 6.99328518, 6.81153822, 7.01038599, 7.54367399]),
 'score_time': array([0.33041096, 0.28047895, 0.28817201, 0.27961302, 0.2831881 ]),
 'test_score': array([0.48666667, 0.57333333, 0.55666667, 0.565     , 0.52      ])}

In [ ]:
np.mean(cv_results['test_score'])

0.5403333333333333

## FIT, PREDICT

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train.drop(TARGETS, axis=1), train[TARGETS], test_size=0.2, random_state=7)

In [ ]:
meta_model = meta_model.fit(
    X=X_train,
    Y=y_train
)

In [ ]:
pred_proba = meta_model.predict_proba(X_test)

In [ ]:
tresholds = get_tresholds(y_test, pred_proba)

0.6630590087148125


In [ ]:
# 0.6689305241201031

In [ ]:
tresholds

{'Сердечная недостаточность': 0.08819127671650837,
 'Стенокардия, ИБС, инфаркт миокарда': 0.10572334976247379,
 'Прочие заболевания сердца': 0.08166613535728656,
 'ОНМК': 0.041910018039294776,
 'Артериальная гипертензия': 0.4662347426296741}

In [ ]:
sample_submission = pd.read_csv('data/original/sample_solution.csv').set_index('ID')

In [ ]:
def make_prediction(test: pd.DataFrame, model: BaseEstimator, tresholds: dict, sample_submission: pd.DataFrame) -> pd.DataFrame:
    assert all(test.index == sample_submission.index)
    
    y_pred = model.predict_proba(test)

    y_pred = np.hstack([pred[:, 1].reshape(-1, 1) for pred in y_pred])
    y_pred = pd.DataFrame(data=y_pred, index=test.index, columns=TARGETS)
    
    for col in TARGETS:
        sample_submission[col] = np.where(y_pred[col].values >= tresholds[col], 1, 0)
        
    return sample_submission


In [ ]:
pred_proba_test = make_prediction(test, meta_model, tresholds, sample_submission)

In [ ]:
sample_submission

,Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца
ID,,,,,
54-001-019-01,1,1,0,1,1
54-002-133-01,1,0,1,1,1
54-001-007-01,0,0,1,1,1
54-102-116-01,0,0,0,0,0
54-502-005-02,1,1,1,1,0
...,...,...,...,...,...
54-102-095-01,1,1,1,0,1
54-102-235-01,1,0,1,1,1
54-502-016-01,1,0,0,1,1


In [ ]:
sample_submission.to_csv('submissions/base_ensemble.csv')